In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

<h4> 
title: "INET4061_Lab3 Jupyter Notebook" <br> 
author: "Sanjana Jonnalagadda" <br> 
date: "September 29, 2018" <br> 
</h4> 

<font color="#581845">
<h1 align="center"> Overview</h1>
    
<h3 align="left"> 
1. Problem: </h3> 
<h4> 
The objective of this notebook is to use a one-way ANOVA test to identify how a random dataset of voters races correspond to their average ages. It is to see whether the voter age means among races are equal or if they differ and if so by how much is there a difference among age mean values among races. </h4> 


<h3 align="left"> 2. References:  </h3>
 <ol> 
    <uli> http://hamelg.blogspot.com/2015/11/python-for-data-analysis-part-16_23.html</uli>
    <br> <br> 
    <uli> http://statisticsbyjim.com/anova/f-tests-anova/ </uli>
    <br> <br> 
    <uli> https://statistics.laerd.com/spss-tutorials/one-way-anova-using-spss-statistics.php </uli>
    <br><br>
 </ol> 

</font>

<font color="#581845">
<h1 align="center"> Data </h1>

<h4> The dataset generates random data of voter races with corresponding voter ages. This data can be used to compare the mean of veter-age pairs using a one-way ANOVA test to illustrate which ages among each race usually votes. </h4> 
<br>
<h4> A one-way ANOVA test is used to determine whether two or more independent groups have any statistically significant differences between the groups means. The ANOVA test uses F-tests to identify if there exists a statitsically significatn diffence between the means of the independent groups. An F-test is based on F-statistic to test the hypothesis. An F-statistic conveys the ratio of two variances. </h4> 
<br>
<h4> The null hypothesis is to assume that the voter age means are equal among all voter race groups. </h4> 

</font> 

In [15]:
np.random.seed(12)

# Voter Races 
races =   ["asian","black","hispanic","other","white"]

# Generate random data
voter_race = np.random.choice(a= races,
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

voter_age = stats.poisson.rvs(loc=18,
                              mu=30,
                              size=1000)

# Group age data by race
voter_frame = pd.DataFrame({"race":voter_race,"age":voter_age})
groups = voter_frame.groupby("race").groups

# Etract individual groups
asian = voter_age[groups["asian"]]
black = voter_age[groups["black"]]
hispanic = voter_age[groups["hispanic"]]
other = voter_age[groups["other"]]
white = voter_age[groups["white"]]

<font color="#581845">
      <h1 align="center"> Explatory Data Analysis </h1>
</font> 

<h4> The F statistic of the ANOVA test shows an F statistic value of 1.77. This illustrates that the mean age among voters of differing races (asian, black, hispanic, other, white) are not the same as the F statistic value does not equal to 1. </h4> 

In [16]:
# Perform the ANOVA
stats.f_oneway(asian, black, hispanic, other, white)

F_onewayResult(statistic=1.7744689357329695, pvalue=0.13173183201930463)

<h4> Another dataset of random voter races was populated. This time the age distributin for white ages was different. The ANOVA test indicates that the F statistic is 10.16 for this distribution. This illustrates that the mean age among voters of different races are still not the same as the F statistic value does not equal to 1. </h4>

In [17]:
np.random.seed(12)

# Generate random data
voter_race = np.random.choice(a= races,
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

# Use a different distribution for white ages
white_ages = stats.poisson.rvs(loc=18, 
                              mu=32,
                              size=1000)

voter_age = stats.poisson.rvs(loc=18,
                              mu=30,iu
                              size=1000)

voter_age = np.where(voter_race=="white", white_ages, voter_age)

# Group age data by race
voter_frame = pd.DataFrame({"race":voter_race,"age":voter_age})
groups = voter_frame.groupby("race").groups   

# Extract individual groups
asian = voter_age[groups["asian"]]
black = voter_age[groups["black"]]
hispanic = voter_age[groups["hispanic"]]
other = voter_age[groups["other"]]
white = voter_age[groups["white"]]

# Perform the ANOVA
stats.f_oneway(asian, black, hispanic, other, white)

SyntaxError: invalid syntax (<ipython-input-17-e2505122287c>, line 15)

<h4> Illustrated below are t-tests that have been conducted to show relationships between the voters races. The p-values below suggest how the mean age of white voters differs in comparison to ther race groups.There may be an errors that arise from using unadjusted pairwaise t-test that overestimate the significance. To account for this possible error, the Bonferroni correction can be applied for multiple comparisons. </h4>

<h4> What is The Bonferroni Correction? <br> <br> The Bonferroni Correction is a method for solving the problems associated with multiple comparisons. The method compensates for the increase in Type 1 error as a rare event is more likely when observed null hypthesis is low and there are multiple hypothesis being tested. It is compensated by testing each individual hypothesis at a significance level of alpha divided by m. Alpha is the desired overall alpha and the m is the number of hypothesis that are there. </h4> 

In [ ]:
# Get all race pairs
race_pairs = []

for race1 in range(4):
    for race2  in range(race1+1,5):
        race_pairs.append((races[race1], races[race2]))

# Conduct t-test on each pair
for race1, race2 in race_pairs: 
    print(race1, race2)
    print(stats.ttest_ind(voter_age[groups[race1]], 
                          voter_age[groups[race2]]))  

<h4> Another method to account for the multiple comparison issue with having an increase in Type 1 error is the Turkey test. </h4> 

<h4>What is the turkey test? <br><br> The turkey test is used to determine which means amongst a set of means differ from the rest. The test compares all possible pairs of means. It uses the formula: Mi-Mj/(sqrt(MS/n)). The numerator of Mi-Mj indicates the difference between the pairs of means. The demoninator is the square root of the mean square within (MS) divided by the number of groups (n).</h4>

<h4>What is the output of the turkey test in this dataset? <br><br> The turkey test indicates in this dataset how the null hypothesis can be rejected for three pairs that include the "white" race category. This shows that the "white" race is different from the other groups. However, the "asian" race groups confidence interval overlaps with the "white" race groups.</h4>

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(endog=voter_age,     # Data
                          groups=voter_race,   # Groups
                          alpha=0.05)          # Significance level

tukey.plot_simultaneous()    # Plot group confidence intervals
plt.vlines(x=49.57,ymin=-0.5,ymax=4.5, color="red")

tukey.summary()              # See test summary

<font color="#581845">
<h1 align="center"> Conclusion </h1>

<h4> In conclusion, the ANOVA test illustrates whether the mean ages of voters varies amongs different race catergory groups. It is seen that the null hypthesis is rejected as the ANOVA test shows that the F statistic is not equal to 1 for the different race catergories to see whether the mean ages are the same. 

<br> <br> 
<ol> 
    <uli> This lab demonstrates using Jupyter to identify: </uli> 
    <br> 
        <uli> - Generate random data as input for a test </uli> 
    <br> 
        <uli> - Compare the mean across levels of one categorical variable using a one-way ANOVA test </uli> 
    
</ol> 


</h4> 
</font> 